In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split


def test_train_split(data, train_ratio = 0.8):
    train_size = int(len(data) * train_ratio)
    test_size = len(data) - train_size
    train_dataset, test_dataset = random_split(data, [train_size, test_size])
    return train_dataset, test_dataset